# Timer

In [3]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'LinearReg']
indicator_list = pattern
indicator_list_higher = ['LinearReg']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': np.arange(2, 102, 2), 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 50


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [06:57<00:00,  8.35s/it]


# Check local statistics

In [3]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,mar_avg,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
5,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,30,0.999733,8.812917,-0.008000,264.387500,8.812917,264.387500,1.2721,1.0724,1.0076,0.9930,0.8236,0.8700,0.9078,1.0110,1.0616,1.0146,1.0565,1.0316,1.0312,1.0235,0.9908,0.9576,0.9380,0.9468,0.9822,0.9382,0.9635,1.0039,1.0282,1.0679,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.35,8.86,9.40,9.96,10.00,10.00,10.00,10.00,10.00,10.00,7.51,7.53,7.57,6.75,6.85,6.96
3,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,15,2.727329,8.549167,25.909938,128.237500,8.549167,128.237500,2.3028,2.0517,2.3330,2.5164,1.9885,2.1540,2.3017,2.7867,3.0776,3.2689,3.6432,3.2758,3.2758,3.3322,3.2664,2.7141,2.6460,2.5949,2.6229,2.5756,2.6329,2.6353,2.6684,2.7911,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.35,8.86,9.40,7.49,7.79,8.01,8.23,8.46,8.76,9.15,7.62,7.96,8.05,8.40,8.43,8.45
4,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,26,1.244413,8.408750,6.354725,218.627500,8.408750,218.627500,1.6604,1.4011,1.3701,1.2868,1.0522,1.1149,1.1469,1.2399,1.2951,1.2361,1.3473,1.3102,1.2992,1.2945,1.2403,1.1821,1.1533,1.1623,1.1965,1.1265,1.1553,1.1643,1.1928,1.2378,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.35,8.86,9.40,7.49,7.79,8.01,8.23,8.46,8.76,9.15,7.62,7.96,8.05,7.21,7.30,7.40
6,STOCH_RSI_LinearReg,14,35,9,7,3,25,14,0,43,1.203254,8.197083,8.739929,352.474583,8.197083,352.474583,1.3136,1.1683,1.0704,1.0358,1.2245,1.2459,1.2661,1.3165,1.3278,1.2911,1.2416,1.2141,1.2145,1.1889,1.1388,1.1025,1.0836,1.1599,1.1829,1.1371,1.1983,1.2284,1.2497,1.2778,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.76,7.35,7.68,8.02,8.05,8.06,8.08,8.12,8.48,8.52,7.36,7.43,7.54,7.05,7.17,7.29
12,STOCH_RSI_LinearReg,14,35,9,7,3,25,26,0,110,0.887383,7.372083,-12.387833,810.929167,7.372083,810.929167,1.0020,0.8848,0.7739,0.7661,0.8951,0.9349,0.9415,0.9521,0.9465,0.8686,0.8560,0.8530,0.8562,0.8502,0.8414,0.8306,0.8159,0.8682,0.9071,0.8887,0.9101,0.9405,0.9559,0.9579,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.76,6.13,6.41,6.70,6.73,6.74,6.76,6.78,7.28,7.33,5.78,5.85,5.96,5.87,5.99,6.09
11,STOCH_RSI_LinearReg,14,35,9,7,3,25,24,0,100,0.956242,7.372083,-4.375833,737.208333,7.372083,737.208333,1.0717,0.9549,0.8344,0.8372,0.9778,0.9809,0.9943,1.0142,1.0200,0.9552,0.9409,0.9315,0.9356,0.9265,0.9141,0.8995,0.8807,0.9386,0.9658,0.9432,0.9716,1.0064,1.0233,1.0315,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.76,6.13,6.41,6.70,6.73,6.74,6.76,6.78,7.28,7.33,5.78,5.85,5.96,5.87,5.99,6.09
9,STOCH_RSI_LinearReg,14,35,9,7,3,25,20,0,83,1.024946,7.372083,2.070504,611.882917,7.372083,611.882917,1.4182,1.1003,0.9215,0.8895,1.0110,1.0113,1.0214,1.0447,1.0493,1.0336,1.0081,0.9804,0.9872,0.9665,0.9556,0.9383,0.9130,0.9803,1.0185,0.9945,1.0374,1.0842,1.1122,1.1217,10.0,10.00,10.00,10.00,10.00,6.81,7.14,7.82,8.76,6.13,6.41,6.70,6.73,6.74,6.76,6.78,7.28,7.33,5.78,5.85,5.96,5.87,5.99,6.09
10,STOCH_RSI_LinearReg,14,35,9,7,3,25,22,0,90,0.986567,7.372083,-1.209000,663.487500,7.372083,663.487500,1.3605,1.0377,0.8932,0.8926,1.0241,1.0013,1.0111,1.0330,1.0348,0.9588,0.9424,0.9304,0.9350,0.9283,0.9159,0.9003,0.8797,0.9393,0.9737,0.9530,0.9872,1.0288,1.0537,1.0628,10.0,10.00,10.00,10.00,10.00,6.8

# Research parameter influence

In [81]:
param = 'LinearReg_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,LinearReg_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,1.937883,-0.562604,65.569898,-39.616042,247
1,3,2.747457,-1.018333,76.114432,-44.385000,123
2,4,4.568197,-2.099375,56.411997,-33.292188,91
3,5,3.548132,-1.371806,74.680826,-39.263194,80
4,6,3.757032,-1.289444,80.305944,-37.298889,80
5,8,3.517467,-1.098056,77.010065,-33.492500,86


# Plot result in file

In [77]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'LinearReg_timeperiod', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

# Save new config data to config file

In [18]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            